In [1]:
import napari
from skimage.data import astronaut
from napari_picasso.main_widget import PicassoWidget
from pyseq import image_analysis as ia

from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift
import numpy as np

#image_path = 'Z:\\gpfs\\commons\\groups\\nygcfaculty\\PySeq\\20210323_4i4color\\zarrs\\m4_o25933_labels.zarr'
#Z:\groups\nygcfaculty\PySeq\20210323_4i4color\zarrs
image_path = 'Z:\\groups\\nygcfaculty\\PySeq\\20210323_4i4color\\zarrs\\m4.zarr'
o = 25933
im = ia.get_HiSeqImages(image_path)
#cropped_im = im.im.sel(row = slice(3800,4000), col = slice(4100, 4300), cycle=5, obj_step=o)
cropped_im = im.im.sel(row = slice(1600,1600+256), col = slice(5000, 5000+256), cycle=5, obj_step=o)
#cropped_im = im.im.sel(cycle=5, obj_step=o)


# # subpixel registration
nuclei = cropped_im.sel(channel=558).compute()
gfap = cropped_im.sel(channel=610).compute()

# subpixel precision
shift, error, diffphase = phase_cross_correlation(gfap, nuclei, upsample_factor=100)


# The shift nuclei image
offset_image = fourier_shift(np.fft.fftn(nuclei), shift)
offset_image = np.fft.ifftn(offset_image)
nuclei_shifted = np.array(offset_image.real).clip(0,4095).astype('int16')



viewer = napari.Viewer()
#widget = PicassoWidget(viewer)
widget = PicassoWidget(viewer)
viewer.window.add_dock_widget(widget)
#viewer.add_image(astronaut(), channel_axis=-1)
# viewer.add_image(cropped_im.sel(channel=558), name='Laminin1b', blending = 'additive', contrast_limits = (0,4095))
# viewer.add_image(cropped_im.sel(channel=610), name='GFAP', blending = 'additive',  contrast_limits = (0,4095))
# viewer.add_image(im.im.sel(obj_step = o, channel=687, cycle=5), name='MBP', blending = 'additive')
# viewer.add_image(im.im.sel(obj_step = o, channel=740, cycle=5), name='ELAVL2', blending = 'additive')

viewer.add_image(nuclei_shifted, name='Laminin1b', blending = 'additive', contrast_limits = (0,4095))
viewer.add_image(gfap, name='GFAP', blending = 'additive',  contrast_limits = (0,4095))


ModuleNotFoundError: No module named 'napari'

In [3]:
# Whole section

shift = [1.25, 0.51]
nuclei = im.im.sel(cycle=5, obj_step=o, channel=558)
gfap = im.im.sel(cycle=5, obj_step=o, channel=610)

# The shift nuclei image
offset_image = fourier_shift(np.fft.fftn(nuclei), shift)
offset_image = np.fft.ifftn(offset_image)
nuclei_shifted = np.array(offset_image.real).clip(0,4095).astype('int16')

viewer = napari.Viewer()
viewer.add_image(nuclei_shifted, name='Laminin1b', blending = 'additive', contrast_limits = (0,4095))
viewer.add_image(gfap, name='GFAP', blending = 'additive',  contrast_limits = (0,4095))


<Image layer 'GFAP' at 0x23c1201a3a0>

C:\Users\kpandit\.conda\envs\cuda\lib\site-packages\napari\plugins\_plugin_manager.py:469: UserWarning: Plugin 'napari-curtain' has already registered a dock widget 'curtain' which has now been overwritten
  warn(message=warn_message)


In [4]:
9187*10240

94074880

In [3]:
im.im.sel(cycle=5, obj_step=o, channel=[558,610])

<xarray.DataArray 'm4' (channel: 2, row: 9187, col: 10240)>
dask.array<getitem, shape=(2, 9187, 10240), dtype=uint16, chunksize=(1, 9187, 2048), chunktype=numpy.ndarray>
Coordinates:
  * channel   (channel) int64 558 610
    cycle     int64 5
    obj_step  int64 25933
Dimensions without coordinates: row, col
Attributes:
    first_group:  0
    machine:      None
    scale:        1
    fixed_bg:     0

In [2]:
widget.sink_sources_match(widget.picasso_params, widget.mixing_dict)
print(widget.sink_sources_dict(widget.picasso_params))
print(widget.sink_sources_dict(widget.mixing_dict))
set(widget.mixing_dict['GFAP'].keys())
widget.picasso_params[0,:,:]

TypeError: unhashable type: 'numpy.ndarray'

In [8]:
test = widget._options.update({})
test.get('max_iter',100)

AttributeError: 'NoneType' object has no attribute 'get'

In [4]:
from napari.utils.notifications import show_info
show_info('Hello Napari')

In [7]:
show_info('Hello Napari')

In [1]:
import torch
import numpy as np
from dask.utils import format_bytes
from picasso.nn_picasso import PICASSOnn
from pyseq import image_analysis as ia


image_path = 'Z:\\gpfs\\commons\\groups\\nygcfaculty\\PySeq\\20210323_4i4color\\zarrs\\m4.zarr'
im = ia.get_HiSeqImages(image_path)
o = 25933
mixed_im = im.im.sel(cycle=5, obj_step=o)

def report_mem():
    reserve_mem = torch.cuda.memory_reserved()
    free_mem = torch.cuda.memory_reserved() - torch.cuda.memory_allocated()
    print('reserved_memory','::',format_bytes(reserve_mem))
    print('free_memory','::',format_bytes(free_mem))
    
report_mem()
mm = np.array([[1],[-1],[-1],[-1]])
model = PICASSOnn(mm)
report_mem()
# dataset = model.get_dataset(mixed_im.sel(channel=[558, 610]))
# report_mem()


C:\Users\kpandit\.conda\envs\cuda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\users\kpandit\pyseq2500\pyseq\image_analysis.py:955: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  im = xr.open_zarr(fn).to_array()


ImageAnalysis::Opened m4 
reserved_memory :: 0 B
free_memory :: 0 B
Using cuda
reserved_memory :: 2.00 MiB
free_memory :: 1.99 MiB


In [7]:
from math import log, floor

2**floor(log(torch.cuda.get_device_properties('cuda').total_memory/8000,2))


262144

In [2]:
dataset = model.get_dataset(mixed_im)
report_mem()

reserved_memory :: 2.00 MiB
free_memory :: 1.99 MiB


In [13]:
from torch.utils.data import DataLoader
import dask.array as da

DA_TYPE = type(da.zeros(0))
TT_TYPE = type(torch.tensor(0))
# def collate(chunk_, dtype = torch.float32):

#     _type = type(chunk_)

#     assert _type in [DA_TYPE, list], f'Expected dask array or list, got {_type}'
#     if _type is list:
#         assert isinstance(chunk_[0], DA_TYPE)
#         chunk_ = chunk_[0]

#     #chunk = torch.tensor(chunk_.compute(), dtype=dtype, device = 'cuda')
#     #chunk = chunk.compute()
    
#     worker_info = torch.utils.data.get_worker_info()
#     if worker_info is None:
#         return iter([None]*3)
#     else:
#     # in a worker process
#     # split workload
#         worker_id = worker_info.id
#         return iter([worker_id]*3)

def collate(chunk_, dtype = torch.float32):
    
    device = 'cuda'
    iscuda = device == 'cuda'

    _type = type(chunk_)

    assert _type in [DA_TYPE, TT_TYPE, list], f'Expected dask array or list, got {_type}'
    if _type is list:
        _type = type(chunk_[0])
        assert _type in [DA_TYPE, TT_TYPE]
        chunk_ = chunk_[0]

    if _type is TT_TYPE:
        if chunk_.device.type != device:
            chunk_ = chunk_.to(device, non_blocking = iscuda)
        return chunk_
    else:
        return torch.tensor(chunk_.compute(), dtype=dtype, device=device)
    

    #return chunk
    
dataloader = DataLoader(dataset, shuffle=True, collate_fn = collate)

for i, im in enumerate(dataloader):
    print(i)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
from 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [3]:

train_gen = model.train_loop(mixed_im)
next(train_gen)
report_mem()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


KeyboardInterrupt: 

In [11]:
test = torch.from_numpy(np.array(0))
test = test.pin_memory()
test.is_pinned()

True

In [ ]:
da_tensor = torch.tensor(cropped_im.sel(channel=558).data.astype('int16').compute(), device = 'cpu')

In [13]:
test.device

NameError: name 'test' is not defined

In [8]:
from torch.utils.data import DataLoader
import dask.array as da


DA_TYPE = type(da.zeros(0))
def dask_collate(chunk_, dtype = torch.float32):

    _type = type(chunk_)

    assert _type in [DA_TYPE, list], f'Expected dask array or list, got {_type}'
    if _type is list:
        assert isinstance(chunk_[0], DA_TYPE)
        chunk_ = chunk_[0]
        
    chunk = torch.tensor(chunk_.compute(), dtype=dtype, device = 'cuda')


    return chunk

dataloader = DataLoader(dataset, shuffle=True, collate_fn = dask_collate)

for batch, im_list in enumerate(dataloader):
    print(batch, im_list.shape)


0 torch.Size([18749440, 2])
1 torch.Size([18831360, 2])
2 torch.Size([18831360, 2])
3 torch.Size([18831360, 2])
4 torch.Size([18831360, 2])


In [5]:
report_mem()

reserved_memory :: 1.27 GiB
free_memory :: 1.99 MiB


In [49]:
00

94074880

In [1]:
import torch
from dask.utils import format_bytes
torch.cuda.memory_reserved() - torch.cuda.memory_allocated()
print(torch.cuda.memory_reserved())
torch.cuda.get_device_properties('cuda')
mixed_im = im.im.sel(cycle=5, obj_step=o)

def report_mem(step):
    free_mem = torch.cuda.memory_reserved() - torch.cuda.memory_allocated()
    print(step,'::',format_bytes(free_mem))
    
report_mem('initializing cuda')
mm = np.array([[1],[-1]])
model = PICASSOnn(mm)
report_mem('creating model')
dataset = model.get_dataset(mixed_im.sel(channel=[558, 610]))
report_mem('creating dataset')

C:\Users\kpandit\.conda\envs\cuda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0


NameError: name 'im' is not defined

In [10]:
import numpy as np

for i, l in enumerate(viewer.layers):
    if l.name == 'unmixed_GFAP':
        print(i)
        break
    
viewer.layers[i].data = np.ones_like(viewer.layers[i].data)



2


In [8]:
import numpy as np
np.ones_like(viewer.layers[i].data)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [13]:
import napari
import numpy as np
from napari_picasso.main_widget import PicassoWidget


# Make sink source images
s = 512; background = 100; signal = 500; alpha = 0.5

sink = np.random.rand(s,s)*10+background
sink[:100, :100] += np.random.rand(100,100)*2000+signal

source = np.zeros((s,s))
source[-100:,-100:] += np.random.rand(100,100)*2000+signal

sink += alpha*source
source += np.random.rand(s,s)*10+background

# Add to viewer
viewer = napari.Viewer(show=False)
viewer.add_image(sink, name='sink')
viewer.add_image(source, name='source')

# setup widget
widget = PicassoWidget(viewer)
widget.add_sink_widget()

sink = widget['sink0']
sink.sink_list.bind(sink)
sink.show_sources()
sink.update_mixing_params()

for s in widget.sinks:
    sink = widget[f'sink{s}']

In [ ]:
from magicgui.widgets import RadioButtons, Container, CheckBox


